# Modelagem para Detecção de Fraudes

Este notebook demonstra o desenvolvimento e avaliação de modelos de machine learning para detecção de fraudes transacionais.

## Objetivos
- Preprocessar dados para modelagem
- Treinar modelos supervisionados e não supervisionados
- Avaliar performance dos modelos
- Comparar diferentes abordagens
- Interpretar resultados

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys
import os

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, average_precision_score
import joblib

# Adiciona o diretório raiz ao path
sys.path.append('..')

# Importações do projeto
from src.data.data_loader import DataLoader
from src.data.preprocessor import DataPreprocessor
from src.models.supervised_models import SupervisedModels
from src.models.unsupervised_models import UnsupervisedModels
from src.utils.helpers import format_percentage

# Configurações
warnings.filterwarnings('ignore')
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('default')
pd.set_option('display.max_columns', None)

print("✅ Bibliotecas importadas com sucesso!")

## Carregamento e Preprocessamento dos Dados

In [ ]:
# Carrega dados
print("Carregando dados...")
loader = DataLoader()
users_df, transactions_df = loader.load_synthetic_data()

print(f"Dados carregados:")
print(f"   - Usuários: {len(users_df):,}")
print(f"   - Transações: {len(transactions_df):,}")
print(f"   - Taxa de fraude: {transactions_df['is_fraud'].mean():.2%}")

In [ ]:
# Preprocessamento
print("Preprocessando dados...")
preprocessor = DataPreprocessor()
processed_df = preprocessor.fit_transform(transactions_df)

print(f"Dados processados:")
print(f"   - Shape original: {transactions_df.shape}")
print(f"   - Shape processado: {processed_df.shape}")
print(f"   - Features criadas: {processed_df.shape[1] - transactions_df.shape[1]}")

In [ ]:
# Divisão treino/teste
X_train, X_test, y_train, y_test = preprocessor.prepare_train_test_split(processed_df)

print(f"Divisão dos dados:")
print(f"   - Treino: {X_train.shape[0]:,} amostras")
print(f"   - Teste: {X_test.shape[0]:,} amostras")
print(f"   - Features: {X_train.shape[1]}")
print(f"   - Taxa de fraude (treino): {y_train.mean():.2%}")
print(f"   - Taxa de fraude (teste): {y_test.mean():.2%}")

## Modelos Supervisionados

In [ ]:
# Inicializa e treina modelos supervisionados
print("Treinando modelos supervisionados...")
supervised_models = SupervisedModels()

# Treina com balanceamento SMOTE
training_results = supervised_models.train_models(
    X_train, y_train, 
    balance_method='smote',
    use_cross_validation=True
)

print("\nResultados do treinamento:")
for model_name, results in training_results.items():
    if 'error' not in results:
        print(f"   - {model_name}: CV Score = {results['cv_mean']:.4f} (+/- {results['cv_std']:.4f})")
    else:
        print(f"   - {model_name}: ERRO - {results['error']}")

In [ ]:
# Avalia modelos no conjunto de teste
print("Avaliando modelos no conjunto de teste...")
evaluation_results = supervised_models.evaluate_models(X_test, y_test)

# Cria DataFrame com resultados
results_data = []
for model_name, metrics in evaluation_results.items():
    if 'error' not in metrics:
        results_data.append({
            'Model': model_name,
            'Accuracy': metrics['accuracy'],
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1-Score': metrics['f1_score'],
            'ROC-AUC': metrics.get('roc_auc', 'N/A')
        })

results_df = pd.DataFrame(results_data)
print("\nMétricas de Avaliação:")
print(results_df.round(4))

In [ ]:
# Visualização das métricas
if len(results_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Precision
    results_df.plot(x='Model', y='Precision', kind='bar', ax=axes[0,0], color='skyblue')
    axes[0,0].set_title('Precision por Modelo')
    axes[0,0].set_ylabel('Precision')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Recall
    results_df.plot(x='Model', y='Recall', kind='bar', ax=axes[0,1], color='lightcoral')
    axes[0,1].set_title('Recall por Modelo')
    axes[0,1].set_ylabel('Recall')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # F1-Score
    results_df.plot(x='Model', y='F1-Score', kind='bar', ax=axes[1,0], color='lightgreen')
    axes[1,0].set_title('F1-Score por Modelo')
    axes[1,0].set_ylabel('F1-Score')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # ROC-AUC (apenas para modelos que têm essa métrica)
    roc_data = results_df[results_df['ROC-AUC'] != 'N/A'].copy()
    if len(roc_data) > 0:
        roc_data['ROC-AUC'] = pd.to_numeric(roc_data['ROC-AUC'])
        roc_data.plot(x='Model', y='ROC-AUC', kind='bar', ax=axes[1,1], color='gold')
        axes[1,1].set_title('ROC-AUC por Modelo')
        axes[1,1].set_ylabel('ROC-AUC')
        axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

# Melhor modelo
try:
    best_model = supervised_models.get_best_model()
    print(f"\nMelhor modelo: {best_model}")
    print(f"   - F1-Score: {evaluation_results[best_model]['f1_score']:.4f}")
    print(f"   - Precision: {evaluation_results[best_model]['precision']:.4f}")
    print(f"   - Recall: {evaluation_results[best_model]['recall']:.4f}")
except:
    print("\n❌ Nenhum modelo foi treinado com sucesso")

## Análise de Curvas ROC e Precision-Recall

In [ ]:
# Curvas ROC e Precision-Recall
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Curva ROC
for model_name in supervised_models.trained_models.keys():
    try:
        y_pred_proba = supervised_models.predict(X_test, model_name, return_probabilities=True)
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        auc_score = roc_auc_score(y_test, y_pred_proba)
        
        axes[0].plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.3f})')
    except Exception as e:
        print(f"Erro ao plotar ROC para {model_name}: {e}")

axes[0].plot([0, 1], [0, 1], 'k--', label='Random')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('Curva ROC')
axes[0].legend()
axes[0].grid(True)

# Curva Precision-Recall
for model_name in supervised_models.trained_models.keys():
    try:
        y_pred_proba = supervised_models.predict(X_test, model_name, return_probabilities=True)
        precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
        ap_score = average_precision_score(y_test, y_pred_proba)
        
        axes[1].plot(recall, precision, label=f'{model_name} (AP = {ap_score:.3f})')
    except Exception as e:
        print(f"Erro ao plotar PR para {model_name}: {e}")

baseline = y_test.mean()
axes[1].axhline(y=baseline, color='k', linestyle='--', label=f'Baseline ({baseline:.3f})')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Curva Precision-Recall')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## Modelos Não Supervisionados

In [ ]:
# Treina modelos não supervisionados
print("🔍 Treinando modelos não supervisionados...")
try:
    unsupervised_models = UnsupervisedModels()
    
    # Remove target para treino não supervisionado
    X_unsupervised = X_train.copy()
    
    # Treina todos os modelos
    unsupervised_results = unsupervised_models.train_all_models(X_unsupervised)
    
    print("\n📊 Resultados dos modelos não supervisionados:")
    for model_name, results in unsupervised_results.items():
        if 'error' not in results:
            anomaly_rate = results.get('anomaly_rate', 0)
            print(f"   - {model_name}: Taxa de anomalias = {anomaly_rate:.2%}")
        else:
            print(f"   - {model_name}: ERRO - {results['error']}")
            
except Exception as e:
    print(f"❌ Erro ao treinar modelos não supervisionados: {e}")
    unsupervised_models = None

In [ ]:
# Avalia modelos não supervisionados com labels (se disponível)
if unsupervised_models is not None:
    try:
        print("📈 Avaliando modelos não supervisionados com labels...")
        unsupervised_evaluation = unsupervised_models.evaluate_with_labels(X_test, y_test)
        
        # Cria DataFrame com resultados
        unsup_results_data = []
        for model_name, metrics in unsupervised_evaluation.items():
            if 'error' not in metrics:
                unsup_results_data.append({
                    'Model': model_name,
                    'Precision': metrics['precision'],
                    'Recall': metrics['recall'],
                    'F1-Score': metrics['f1_score'],
                    'Anomaly Rate': metrics['anomaly_rate']
                })
        
        if unsup_results_data:
            unsup_results_df = pd.DataFrame(unsup_results_data)
            print("\n📊 Métricas dos Modelos Não Supervisionados:")
            print(unsup_results_df.round(4))
        else:
            print("\n❌ Nenhum modelo não supervisionado foi avaliado com sucesso")
            
    except Exception as e:
        print(f"❌ Erro na avaliação: {e}")

## 🎯 Importância das Features

In [ ]:
# Análise de importância das features (para modelos que suportam)
try:
    best_model = supervised_models.get_best_model()
    feature_names = X_train.columns.tolist()
    
    feature_importance = supervised_models.get_feature_importance(best_model, feature_names)
    
    if feature_importance:
        # Top 15 features mais importantes
        top_features = dict(list(feature_importance.items())[:15])
        
        plt.figure(figsize=(12, 8))
        features = list(top_features.keys())
        importances = list(top_features.values())
        
        plt.barh(range(len(features)), importances)
        plt.yticks(range(len(features)), features)
        plt.xlabel('Importância')
        plt.title(f'Top 15 Features Mais Importantes - {best_model}')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        
        print(f"\n🎯 Top 10 features mais importantes ({best_model}):")
        for i, (feature, importance) in enumerate(list(feature_importance.items())[:10]):
            print(f"   {i+1:2d}. {feature}: {importance:.4f}")
    else:
        print(f"\n❌ Modelo {best_model} não suporta análise de importância")
        
except Exception as e:
    print(f"❌ Erro na análise de importância: {e}")

## 💾 Salvando Modelos

In [ ]:
# Salva modelos treinados
print("💾 Salvando modelos...")
try:
    saved_paths = supervised_models.save_models()
    
    print("\n✅ Modelos salvos:")
    for model_name, path in saved_paths.items():
        print(f"   - {model_name}: {path}")
        
except Exception as e:
    print(f"❌ Erro ao salvar modelos: {e}")

# Salva também o preprocessador
try:
    import joblib
    joblib.dump(preprocessor, 'models/preprocessor.pkl')
    print("   - preprocessor: models/preprocessor.pkl")
except Exception as e:
    print(f"❌ Erro ao salvar preprocessador: {e}")

## 📊 Resumo Final

In [ ]:
# Resumo final dos resultados
print("📊 RESUMO FINAL - DETECÇÃO DE FRAUDES")
print("=" * 50)

print(f"\n📥 DADOS:")
print(f"   - Total de transações: {len(transactions_df):,}")
print(f"   - Taxa de fraude: {transactions_df['is_fraud'].mean():.2%}")
print(f"   - Features após preprocessamento: {X_train.shape[1]}")

if len(results_df) > 0:
    print(f"\n🤖 MODELOS SUPERVISIONADOS:")
    best_supervised = results_df.loc[results_df['F1-Score'].idxmax()]
    print(f"   - Melhor modelo: {best_supervised['Model']}")
    print(f"   - F1-Score: {best_supervised['F1-Score']:.4f}")
    print(f"   - Precision: {best_supervised['Precision']:.4f}")
    print(f"   - Recall: {best_supervised['Recall']:.4f}")
    if best_supervised['ROC-AUC'] != 'N/A':
        print(f"   - ROC-AUC: {best_supervised['ROC-AUC']:.4f}")

if 'unsup_results_df' in locals() and len(unsup_results_df) > 0:
    print(f"\n🔍 MODELOS NÃO SUPERVISIONADOS:")
    best_unsupervised = unsup_results_df.loc[unsup_results_df['F1-Score'].idxmax()]
    print(f"   - Melhor modelo: {best_unsupervised['Model']}")
    print(f"   - F1-Score: {best_unsupervised['F1-Score']:.4f}")
    print(f"   - Precision: {best_unsupervised['Precision']:.4f}")
    print(f"   - Recall: {best_unsupervised['Recall']:.4f}")

print(f"\n🎯 PRÓXIMOS PASSOS:")
print(f"   1. Implementar sistema de regras de negócio")
print(f"   2. Criar pipeline de predição em tempo real")
print(f"   3. Desenvolver sistema de monitoramento")
print(f"   4. Implementar retreinamento automático")
print(f"   5. Criar alertas e notificações")

print("\n✅ Modelagem concluída com sucesso!")